In [1]:
%run ./path.ipynb

In [2]:
%run ./setting.ipynb

# load data

In [3]:
ratings = pd.read_parquet(
    os.path.join(
        data_path,
        ratings_with_date,
    )
    )

# movie to movie similarity by ratings

In [4]:
user_movie = sql(f"""
select distinct user_id, movie_id, rating
from ratings
where date < '{test_start_date}'
limit 10000
""")

## common users giving the same rate

In [5]:
%%time

user_movie_pair = sql(f"""
select 
a.user_id,
a.rating,
a.movie_id as movie_id_a,
b.movie_id as movie_id_b
from user_movie as a 
join user_movie as b
on a.user_id = b.user_id 
and a.rating = b.rating
and a.movie_id < b.movie_id
""")

CPU times: total: 672 ms
Wall time: 675 ms


In [6]:
user_movie_pair.head(3)

user_id  rating  movie_id_a  movie_id_b
0        1       5        1193        1270
1        1       5        1193        1287
2        1       5        1193        1836

## count common users

In [7]:
%%time

movie_similarity_by_user_number = sql(f"""
select 
movie_id_a,
movie_id_b,
rating,
count(*) as common_users
from user_movie_pair
group by 
movie_id_a,
movie_id_b,
rating
""")

CPU times: total: 1.2 s
Wall time: 1.3 s


movie_id_a  movie_id_b  rating  common_users
0            1           2       5             1
1            1           4       3             1
2            1           6       4             1
3            1           6       5             1
4            1           7       4             1
..         ...         ...     ...           ...
95           1         339       4             1
96           1         339       5             1
97           1         349       4             2
98           1         350       3             1
99           1         356       5             4

[100 rows x 4 columns]

In [9]:
sql(f"""
select *
from movie_similarity_by_user_number
order by common_users desc 
limit 10;
""")

movie_id_a  movie_id_b  rating  common_users
0         260        1198       5            14
1         260        1196       5            13
2         260        1240       5            10
3         356        1196       5            10
4         260         593       5             9
5         260        1210       5             9
6         356        2762       5             9
7         480        1196       5             9
8         589        2858       4             9
9         593        1196       5             9

In [10]:
sql(f"""
select 
count(*)
from movie_similarity_by_user_number;
""")

count(*)
0    413226

## normalize

In [11]:
%%time

movie_user_number = sql(f"""
select 
movie_id,
rating,
count(*) as users
from user_movie
group by 
movie_id,
rating
""")

CPU times: total: 15.6 ms
Wall time: 21.9 ms
